In [68]:
import pandas as pd
import numpy as np
import pickle as pkl
from scipy.stats import fisher_exact
import requests
path = 'C:/Users/alber/OneDrive/Documenti/universita/DATA SCIENCE/Biological Data/PROJECT/'

with open(path+'Original_dataset.txt','r') as o:
    original_dataset = pd.read_csv(o, header = None)
original_proteins = set(original_dataset[0])

with open(path+'GO_human_dataset','r') as o:
    human_GO_ds = pd.read_csv(o, index_col=0, dtype = str)

with open(path+'ARCHITECTURE/architecture_datasets.pkl','rb') as o:
    arch_datasets = pkl.load(o)

with open(path+'PDB_dataset.txt','r') as o:
    pdb_dataset = pd.read_csv(o, index_col=None, dtype = str, header = None)
    
with open(path+'String_dataset.txt','r') as o:
    string_dataset = pd.read_csv(o, index_col=None, dtype = str, header = None)
    
with open(path+'new_string_dataset.txt','r') as o:
    new_string_dataset = pd.read_csv(o, index_col=None, dtype = str, header = None)
    
with open('pdb_chain_uniprot.txt','r') as o:
    human_pdb_dataset = pd.read_csv(o, index_col=None, dtype = str)
    
human_GO_ds.head()

,Entry,GO_id,Go_description,Length,Pfam_domains
0,Q8TE23,0016021,integral component of membrane,839,PF00003;PF01094;PF07562;
1,Q8TE23,0005887,integral component of plasma membrane,839,PF00003;PF01094;PF07562;
2,Q8TE23,0005886,plasma membrane,839,PF00003;PF01094;PF07562;
3,Q8TE23,0043235,receptor complex,839,PF00003;PF01094;PF07562;
4,Q8TE23,1903767,sweet taste receptor complex,839,PF00003;PF01094;PF07562;


In [14]:
def get_GO_ids(ds, proteins):
    # slice of human ds considering just 'proteins' entries
    original_GO_ds = ds[ds['Entry'].isin(proteins)]
    # put the GO_ids in a set
    original_GO_ids = set(list(original_GO_ds['GO_id'].values))
    # removing nans
    if np.nan in original_GO_ids:
        original_GO_ids.remove(np.nan)
    return original_GO_ids

def get_frequencies(human_GO_ds, GO_ids, proteins):
    sliced_GO_ds = human_GO_ds[human_GO_ds['Entry'].isin(proteins)]
    GO_frequencies = {}
    for GO_id in GO_ids:
        GO_frequencies[GO_id] = len(sliced_GO_ds[sliced_GO_ds['GO_id'] == GO_id])
    return GO_frequencies

def get_all_proteins(human_GO_ds):
    return set(list(human_GO_ds['Entry'].values))

def remove_blank_space(string):
    while string[0] == ' ':
        string = string[1:]
    
    while string[-1] == ' ':
        string = string[:-1]
    return string

def get_description(GO_id, human_GO_ds):
    descs = set(list(human_GO_ds[human_GO_ds['GO_id'] == GO_id]['Go_description'].values))
    final_descs = []
    for d in descs:
        final_descs.append(remove_blank_space(d))
    
    return final_descs[0]



def Fisher_test(n_a, n_b,freq_a,freq_b, GO_ids, human_ds):

    Fisher_test_stats = {}
    Fisher_test_stats['GO_id'] = []
    Fisher_test_stats['Original_freq'] = []
    Fisher_test_stats['Human_freq'] = []
    Fisher_test_stats['Odds_ratio'] = []
    Fisher_test_stats['p-value'] = []
    Fisher_test_stats['GO_description'] = []
    for GO_id in GO_ids:
        mat = np.array([[freq_a[GO_id], n_a - freq_a[GO_id]],
                        [freq_b[GO_id], n_b - freq_b[GO_id]]])
        GO_desc = get_description(GO_id, human_ds)
        odds_ratio,pvalue = fisher_exact(mat)
        Fisher_test_stats['GO_description'].append(GO_desc)
        Fisher_test_stats['GO_id'].append(GO_id)
        Fisher_test_stats['Original_freq'].append(freq_a[GO_id])
        Fisher_test_stats['Human_freq'].append(freq_b[GO_id])
        Fisher_test_stats['Odds_ratio'].append(odds_ratio)
        Fisher_test_stats['p-value'].append(pvalue)
    
    Fisher_results = pd.DataFrame(Fisher_test_stats)
    
    return Fisher_results

### FISHER TEST: ORIGINAL VS HUMAN

In [3]:
human_proteins = get_all_proteins(human_GO_ds)
original_GO_ids =  get_GO_ids(human_GO_ds, proteins = original_proteins)
original_frequences = get_frequencies(human_GO_ds, GO_ids = original_GO_ids, proteins = original_proteins)
# for the human frequences we count the appearences of the same GO_ids also in different proteins
human_frequences = get_frequencies(human_GO_ds, GO_ids = original_GO_ids, proteins = human_proteins)

In [4]:
print(len(human_frequences) == len(original_frequences))

True


In [7]:
Fisher_test(len(original_proteins), len(human_proteins),original_frequences,human_frequences, original_GO_ids, human_GO_ds)

,GO_id,Original_freq,Human_freq,Odds_ratio,p-value,GO_description
0,0001736,2,14,57.011204,0.000775,establishment of planar polarity
1,0060122,1,17,23.020362,0.045721,inner ear receptor cell stereocilium organization
2,0010754,1,6,65.259615,0.018030,negative regulation of cGMP-mediated signaling
3,0006974,1,244,1.585987,0.473004,cellular response to DNA damage stimulus
4,0000785,1,101,3.858720,0.233364,chromatin
...,...,...,...,...,...,...
532,0002091,1,11,35.587413,0.030713,negative regulation of receptor internalization
533,0010596,1,22,17.784091,0.058053,negative regulation of endothelial cell migration
534,0060173,1,31,12.615385,0.079858,limb development
535,0016525,1,86,4.535107,0.202744,negative regulation of angiogenesis


### FISHER TEST: ARCHITECTURE VS ORIGINAL

In [12]:
# in this case we take the proteins from each architecture dataset defined by a different combination of pfam entries
pfam_keys = arch_datasets.keys()
arch_proteins = arch_datasets

In [13]:
arch_GO_ids = {pfam_key: get_GO_ids(human_GO_ds, arch_proteins[pfam_key]) for pfam_key in pfam_keys}
arch_frequences = {pfam_key: get_frequencies(human_GO_ds, arch_GO_ids[pfam_key], arch_proteins[pfam_key]) for pfam_key in pfam_keys}
original_frequences = {pfam_key: get_frequencies(human_GO_ds, arch_GO_ids[pfam_key], original_proteins) for pfam_key in pfam_keys}

Architecture_f_tests = {pfam_key: Fisher_test(len(arch_datasets[pfam_key]), len(original_proteins),
                                  arch_frequences[pfam_key],original_frequences[pfam_key], arch_GO_ids[pfam_key], human_GO_ds)
                                  for pfam_key in pfam_keys}

In [90]:
#with open('Architecture_Fisher_tests.pkl','wb') as save_file:
    #pkl.dump(Architecture_f_tests,save_file)

### FISHER TEST: PDB VS HUMAN

In [6]:
human_pdb_dataset.head()

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,101m,A,P02185,1,154,0,153,1,154
1,102l,A,P00720,1,40,1,40,1,40
2,102l,A,P00720,42,165,41,None,41,164
3,102m,A,P02185,1,154,0,153,1,154
4,103l,A,P00720,1,40,1,None,1,40


In [11]:
# intersection of human swiss and pdb dataset
pdb_human_proteins = set(list(human_pdb_dataset['SP_PRIMARY'].values))
pdb_GO_ds = human_GO_ds[human_GO_ds['Entry'].isin(pdb_human_proteins)] #intersection dataset
pdb_GO_ds.head()



,Entry,GO_id,Go_description,Length,Pfam_domains
49,Q15637,0016604,nuclear body,639,PF00013;PF16275;PF00098;
50,Q15637,0005654,nucleoplasm,639,PF00013;PF16275;PF00098;
51,Q15637,0005634,nucleus,639,PF00013;PF16275;PF00098;
52,Q15637,0005840,ribosome,639,PF00013;PF16275;PF00098;
53,Q15637,0005681,spliceosomal complex,639,PF00013;PF16275;PF00098;


In [16]:
pdb_proteins = set(list(pdb_dataset[0].values))
pdb_GO_ids =  get_GO_ids(pdb_GO_ds, proteins = pdb_proteins)
pdb_frequences = get_frequencies(pdb_GO_ds, GO_ids = pdb_GO_ids, proteins = pdb_proteins)
human_frequences = get_frequencies(pdb_GO_ds, GO_ids = pdb_GO_ids, proteins = pdb_human_proteins)

In [17]:
pdb_fishert_results = Fisher_test(len(pdb_proteins), len(pdb_human_proteins),pdb_frequences,human_frequences, pdb_GO_ids, pdb_GO_ds)

In [18]:
pdb_fishert_results.head()

,GO_id,Original_freq,Human_freq,Odds_ratio,p-value,GO_description
0,0070207,1,24,26.277778,0.039313,protein homotrimerization
1,0051393,1,12,52.568376,0.020637,alpha-actinin binding
2,0005903,3,30,64.718421,0.000021,brush border
3,0043154,1,47,13.412166,0.074132,negative regulation of cysteine-type endopepti...
4,0043555,1,2,315.474359,0.004800,regulation of translation in response to stress


In [19]:
pdb_fishert_results.to_csv('PDB_ds_fresults.txt')

### FISHER TEST: STRING VS HUMAN

##### original string data

In [62]:
# SWISS/STRING intersection
with open(path+'ss_GO_human_dataset.csv','r') as o:
    ss_human_GO_ds = pd.read_csv(o, index_col=0, dtype = str)

ss_human_GO_ds.head()

,Entry,GO_id,Go_description,Length,Pfam_domains
0,P0C7T8,0016021,integral component of membrane,217,NaN
1,Q68DL7,NaN,NaN,685,PF15813;
2,Q9Y6J9,0070062,extracellular exosome,622,PF02969;PF07571;
3,Q9Y6J9,0000118,histone deacetylase complex,622,PF02969;PF07571;
4,Q9Y6J9,0005654,nucleoplasm,622,PF02969;PF07571;


In [63]:
ss_human_proteins = set(list(ss_human_GO_ds['Entry'].values))
string_proteins = set(list(string_dataset[0].values))
string_GO_ids =  get_GO_ids(ss_human_GO_ds, proteins = string_proteins)
string_frequences = get_frequencies(ss_human_GO_ds, GO_ids = string_GO_ids, proteins = string_proteins)
ss_human_frequences = get_frequencies(ss_human_GO_ds, GO_ids = string_GO_ids, proteins = ss_human_proteins)

In [64]:
string_fishert_results = Fisher_test(len(string_proteins), len(ss_human_proteins),string_frequences,ss_human_frequences, string_GO_ids, ss_human_GO_ds)

In [65]:
string_fishert_results.head()

,GO_id,Original_freq,Human_freq,Odds_ratio,p-value,GO_description
0,0009268,1,9,22.405797,0.048763,response to pH
1,1902993,1,3,67.239130,0.019795,positive regulation of amyloid precursor prote...
2,0048704,1,41,4.909862,0.189535,embryonic skeletal system morphogenesis
3,0005903,3,54,11.424074,0.002889,brush border
4,2000553,1,7,28.810559,0.039202,positive regulation of T-helper 2 cell cytokin...


In [66]:
print(len(string_fishert_results))

1156


In [67]:
string_fishert_results.to_csv('String_ds_fresults.txt')

##### new string data

In [70]:
new_string_proteins = set(list(new_string_dataset[0].values))
new_string_GO_ids =  get_GO_ids(ss_human_GO_ds, proteins = new_string_proteins)
new_string_frequences = get_frequencies(ss_human_GO_ds, GO_ids = new_string_GO_ids, proteins = new_string_proteins)
new_ss_human_frequences = get_frequencies(ss_human_GO_ds, GO_ids = new_string_GO_ids, proteins = ss_human_proteins)

In [74]:
new_string_fishert_results = Fisher_test(len(new_string_proteins), len(ss_human_proteins),new_string_frequences,new_ss_human_frequences,
                                         new_string_GO_ids, ss_human_GO_ds)

In [75]:
print(len(new_string_fishert_results))

3818


In [76]:
new_string_fishert_results.to_csv('New_string_ds_fresults.txt')